- https://qdrant.github.io/fastembed/qdrant/Retrieval_with_FastEmbed/#setup

In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
import os
import random
from collections import Counter
from math import ceil

import numpy as np
import torch
from fastembed import TextEmbedding
from fastembed.rerank.cross_encoder import TextCrossEncoder
from gait import FEL, FEL1, FEL2, Layers
from rich.pretty import pprint

In [ ]:
layers = Layers.load("/Users/mraad/data/NorthSea.json").prune_layers()

In [ ]:
fel = FEL(layers=layers)

In [ ]:
pprint(fel.create_line_0(), expand_all=True)

In [ ]:
func = fel.create_line_0

In [ ]:
elems = [func() for _ in range(10_000)]
elems = {_.line: _.fel for _ in elems}

In [ ]:
docs = list(elems.keys())
fels = list(elems.values())

In [ ]:
len(docs)

In [ ]:
for _ in random.choices(docs, k=3):
    print(_)
    print()

In [ ]:
text_emb = TextEmbedding(
    model_name="BAAI/bge-base-en-v1.5",
)

In [ ]:
embeddings = list(text_emb.embed(docs, batch_size=1024, parallel=0))

In [ ]:
pprint(TextCrossEncoder.list_supported_models(), expand_all=True)

In [ ]:
encoder = TextCrossEncoder(model_name="Xenova/ms-marco-MiniLM-L-12-v2")

In [ ]:
prompt = func().line
print(prompt)
embedding = next(iter(text_emb.embed(prompt)))
similarities = np.dot(embeddings, embedding)

In [ ]:
similarities.min(), similarities.max()

In [ ]:
top = 30
indicies = similarities.argsort()[-top:]

In [ ]:
new_docs = [docs[_] for _ in indicies]
new_fels = [fels[_] for _ in indicies]

In [ ]:
new_sims = np.array(list(encoder.rerank(prompt, new_docs)))

In [ ]:
for _ in new_sims.argsort()[-15:]:
    print(new_sims[_])
    pprint(new_fels[_], expand_all=True)